<a href="https://colab.research.google.com/github/Flychuban/Tyres-Condition/blob/main/TyreCondition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import tensorflow as tf
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D

In [44]:
train_path = Path('/content/drive/MyDrive/TyreConditionPrediction/data/Tire Textures/training_data')
test_path = Path('/content/drive/MyDrive/TyreConditionPrediction/data/Tire Textures/testing_data')
RANDOM_SEED = 10
EPOCHS = 10
BATCH_SIZE = 16

In [45]:
labels = os.listdir(train_path)
labels

['normal', 'cracked']

Let's make our dataset

In [46]:
def process_data(data):
  path = Path(data)
  filepaths = list(path.glob(r'*/*.jpg'))
  labels=list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))#Separating the label from filepath and storing it
  df1 = pd.Series(filepaths, name='filepaths').astype(str)
  df2 = pd.Series(labels, name='labels')
  df = pd.concat([df1, df2], axis=1)
  return df

In [47]:
train_data = process_data(train_path)
test_data = process_data(test_path)
train_data

,filepaths,labels
0,/content/drive/MyDrive/TyreConditionPrediction...,normal
1,/content/drive/MyDrive/TyreConditionPrediction...,normal
2,/content/drive/MyDrive/TyreConditionPrediction...,normal
3,/content/drive/MyDrive/TyreConditionPrediction...,normal
4,/content/drive/MyDrive/TyreConditionPrediction...,normal
...,...,...
698,/content/drive/MyDrive/TyreConditionPrediction...,cracked
699,/content/drive/MyDrive/TyreConditionPrediction...,cracked
700,/content/drive/MyDrive/TyreConditionPrediction...,cracked
701,/content/drive/MyDrive/TyreConditionPrediction...,cracked


In [49]:
train_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [51]:
train_images = train_generator.flow_from_dataframe(dataframe=train_data,
                                                   target_size=(224, 224),
                                                   batch_size = BATCH_SIZE,
                                                   subset='training',
                                                   x_col='filepaths',
                                                   y_col='labels',
                                                   random_seed=RANDOM_SEED)

test_images = test_generator.flow_from_dataframe(dataframe=test_data,
                                                 target_size=(224, 224),
                                                 batch_size=BATCH_SIZE,
                                                 x_col='filepaths',
                                                 y_col = 'labels',
                                                 random_seed = RANDOM_SEED
                                                 )

Found 703 validated image filenames belonging to 2 classes.
Found 325 validated image filenames belonging to 2 classes.


Build the model

In [52]:
pretrained_model = ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights = 'imagenet',
    pooling='max'
)

In [53]:
pretrained_model.trainable = False # we make this cuz w don't train again the big model

In [54]:
inputs = pretrained_model.input

new_layer = Dense(120, activation='relu')(pretrained_model.output)
new_layer = Dense(120, activation ='relu')(new_layer)

outputs = Dense(2, activation = 'softmax')(new_layer)

In [55]:
model = Model(inputs=inputs, outputs=outputs)

In [56]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
)

Training

In [57]:
history = model.fit(train_images, epochs=EPOCHS)

Epoch 1/10
44/44 [==============================] - 189s 4s/step - loss: 0.5069 - accuracy: 0.8521
Epoch 2/10
44/44 [==============================] - 165s 4s/step - loss: 0.1412 - accuracy: 0.9502
Epoch 3/10
44/44 [==============================] - 164s 4s/step - loss: 0.1319 - accuracy: 0.9559
Epoch 4/10
44/44 [==============================] - 169s 4s/step - loss: 0.1107 - accuracy: 0.9644
Epoch 5/10
44/44 [==============================] - 166s 4s/step - loss: 0.0459 - accuracy: 0.9872
Epoch 6/10
44/44 [==============================] - 165s 4s/step - loss: 0.0235 - accuracy: 0.9929
Epoch 7/10
44/44 [==============================] - 170s 4s/step - loss: 0.0144 - accuracy: 0.9957
Epoch 8/10
44/44 [==============================] - 170s 4s/step - loss: 0.0036 - accuracy: 1.0000
Epoch 9/10
44/44 [==============================] - 169s 4s/step - loss: 0.0012 - accuracy: 1.0000
Epoch 10/10
44/44 [==============================] - 169s 4s/step - loss: 6.5026e-04 - accuracy: 1.0000
